In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
masoudnickparvar_brain_tumor_mri_dataset_path = kagglehub.dataset_download('masoudnickparvar/brain-tumor-mri-dataset')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers, models, callbacks
from tensorflow.data import AUTOTUNE
from tensorflow.keras.applications import efficientnet, EfficientNetB1

IMG_SIZE=(224,224)
BATCH_SIZE=64
train_path="/kaggle/input/brain-tumor-mri-dataset/Training"
test_path="/kaggle/input/brain-tumor-mri-dataset/Testing"

train_ds = tf.keras.utils.image_dataset_from_directory(
    train_path,
    labels="inferred",
    label_mode="categorical",
    color_mode="rgb",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True,
)

num_classes = len(train_ds.class_names)

test_ds = tf.keras.utils.image_dataset_from_directory(
    test_path, # Corrected from TEST_DIR to test_path
    labels="inferred",
    label_mode="categorical",
    color_mode="rgb",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False
)

augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(15.0/360.0),
    layers.RandomTranslation(0.1, 0.1),
    layers.RandomZoom(0.1, 0.1),
])

preprocess_input = efficientnet.preprocess_input

def preprocess(images, labels):
    images = tf.cast(images, tf.float32)
    images = augmentation(images, training=True)
    images = efficientnet.preprocess_input(images)
    return images, labels

def preprocess_eval(images, labels): # Defined preprocess_eval for test data without augmentation
    images = tf.cast(images, tf.float32)
    images = efficientnet.preprocess_input(images)
    return images, labels

train_ds = train_ds.map(preprocess, num_parallel_calls=AUTOTUNE) # Added num_parallel_calls
train_ds = train_ds.shuffle(2000).cache().prefetch(AUTOTUNE) # Corrected typo train_dstrain_ds to train_ds
test_ds = test_ds.map(preprocess_eval, num_parallel_calls=AUTOTUNE).cache().prefetch(AUTOTUNE)



#Model Building
base_model = EfficientNetB1(include_top=False, input_shape=(*IMG_SIZE, 3), weights="imagenet")
base_model.trainable = True


print(f"Total layers in base model: {len(base_model.layers)}")

fine_tune_at = len(base_model.layers) - 30

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

inputs = tf.keras.Input(shape=(224, 224, 3))
x =(base_model(inputs, training=True))
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
# final Dense with softmax for categorical labels
outputs = layers.Dense(num_classes, activation="softmax")(x)
model = models.Model(inputs, outputs)

model.compile(
    optimizer="Adam",
    loss="categorical_crossentropy",
    metrics=["accuracy", tf.keras.metrics.AUC(name="auc")]
)

#Callbacks
checkpoint_cb = callbacks.ModelCheckpoint("best_model.h5", save_best_only=True, monitor="val_auc", mode="max")
reduce_lr_cb = callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-6)
earlystop_cb = callbacks.EarlyStopping(monitor="val_loss", patience=6, restore_best_weights=True)

history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=10,
    callbacks=[checkpoint_cb, reduce_lr_cb, earlystop_cb]
)

test_loss, test_acc, test_auc = model.evaluate(test_ds)

print(f"Test loss: {test_loss:.4f}, test acc: {test_acc:.4f}, test auc: {test_auc:.4f}")

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

Found 5712 files belonging to 4 classes.


I0000 00:00:1764691278.962001     461 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


Found 1311 files belonging to 4 classes.
Total layers in base model: 340
Epoch 1/10


...

90/90 ━━━━━━━━━━━━━━━━━━━━ 9s 98ms/step - accuracy: 0.9399 - auc: 0.9956 - loss: 0.1531 - val_accuracy: 0.9085 - val_auc: 0.9889 - val_loss: 0.2387 - learning_rate: 0.0010
Epoch 3/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - accuracy: 0.9702 - auc: 0.9987 - loss: 0.0829

90/90 ━━━━━━━━━━━━━━━━━━━━ 9s 97ms/step - accuracy: 0.9702 - auc: 0.9987 - loss: 0.0830 - val_accuracy: 0.9382 - val_auc: 0.9942 - val_loss: 0.1678 - learning_rate: 0.0010
Epoch 4/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 8s 90ms/step - accuracy: 0.9796 - auc: 0.9993 - loss: 0.0584 - val_accuracy: 0.9512 - val_auc: 0.9940 - val_loss: 0.1739 - learning_rate: 0.0010
Epoch 5/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 8s 91ms/step - accuracy: 0.9877 - auc: 0.9996 - loss: 0.0409 - val_accuracy: 0.9283 - val_auc: 0.9850 - val_loss: 0.2839 - learning_rate: 0.0010
Epoch 6/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 8s 90ms/step - accuracy: 0.9829 - auc: 0.9996 - loss: 0.0464 - val_accuracy: 0.9458 - val_auc: 0.9914 - val_loss: 0.1865 - learning_rate: 0.0010
Epoch 7/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - accuracy: 0.9904 - auc: 0.9998 - loss: 0.0325

90/90 ━━━━━━━━━━━━━━━━━━━━ 9s 97ms/step - accuracy: 0.9905 - auc: 0.9998 - loss: 0.0324 - val_accuracy: 0.9558 - val_auc: 0.9946 - val_loss: 0.1489 - learning_rate: 5.0000e-04
Epoch 8/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 8s 90ms/step - accuracy: 0.9976 - auc: 1.0000 - loss: 0.0138 - val_accuracy: 0.9458 - val_auc: 0.9932 - val_loss: 0.1625 - learning_rate: 5.0000e-04
Epoch 9/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - accuracy: 0.9952 - auc: 1.0000 - loss: 0.0130

90/90 ━━━━━━━━━━━━━━━━━━━━ 9s 98ms/step - accuracy: 0.9952 - auc: 1.0000 - loss: 0.0130 - val_accuracy: 0.9603 - val_auc: 0.9949 - val_loss: 0.1302 - learning_rate: 5.0000e-04
Epoch 10/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - accuracy: 0.9985 - auc: 1.0000 - loss: 0.0082

90/90 ━━━━━━━━━━━━━━━━━━━━ 9s 96ms/step - accuracy: 0.9985 - auc: 1.0000 - loss: 0.0083 - val_accuracy: 0.9573 - val_auc: 0.9950 - val_loss: 0.1356 - learning_rate: 5.0000e-04
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - accuracy: 0.9447 - auc: 0.9925 - loss: 0.1766
Test loss: 0.1302, test acc: 0.9603, test auc: 0.9949


In [3]:
model.save("brain_tumor_predictor.keras")